In [1]:
import json
import pyhf
pyhf.set_backend('jax')

import pymc as pm
import arviz as az

import numpy as np

import pytensor
from pytensor import tensor as pt
from pytensor.graph.basic import Apply
from pytensor.graph import Apply, Op

# import aesara
import aesara.tensor as at
# from aesara.graph.op import Op
from aesara.link.jax.dispatch import jax_funcify

import jax
from jax import grad, jit, vmap, value_and_grad, random
import jax.numpy as jnp


# import sys
# sys.path.insert(1, '/Users/malinhorstmann/Documents/pyhf_pymc/src')
import MH_inference
import HMC_inference
import prepare_inference

### Model

In [2]:
# Displaced Leptons
with open('SRee_SRmm_Srem.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
DL_model = workspace.model()
DL_obs = workspace.data(DL_model, include_auxdata=False)
DL_nBins = len(DL_model.expected_actualdata(DL_model.config.suggested_init()))

# ttbar
with open('ttbar_ljets_xsec_inclusive_pruned.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
ttbar_model = workspace.model()
ttbar_obs = workspace.data(ttbar_model, include_auxdata=False)
ttbar_nBins = len(DL_model.expected_actualdata(ttbar_model.config.suggested_init()))

In [4]:
# List with all models
# all_models = [['Dummy_Model', dummy_model, dummy_obs, dummy_nBins],     
all_models = [['DisplacedLeptons', DL_model, DL_obs, DL_nBins], 
              ['ttbar', ttbar_model, ttbar_obs, ttbar_nBins]]

### General Stuff

In [5]:
for name, model, obs, nBins in all_models:
    # Supply information on the unconstrained parameters
    unconstr_dict = {
    'uncon1': {'type': 'unconstrained', 'type2': 'normal', 'input': [[1], [0.1]]}
    }

    # Create dictionary with all priors (unconstrained, constrained by normal and poisson)
    prior_dict = prepare_inference.prepare_priors(model, unconstr_dict)

    # dictionary with keys 'model', 'obs', 'priors', 'precision'
    prepared_model = prepare_inference.prepare_model(model=model, observations=obs, precision=0.10, priors=prior_dict)

### Sampling